# Introduction

This data are for an electronics shop, it contains sales for year 2019, a csv file for each month
each dataset cosists of 6 columns ['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date', 'Purchase Address']
This notebook visualizes the data and answers some questions on the data provided

# Installing Libraries

In [2]:
# uncomment the line associated with any library you don't have

#!pip3 install numpy -q
#!pip3 install pandas -q
#!pip3 install matplotlib -q
#!pip3 install regex -q

# Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import re
%matplotlib inline

# Reading Data

Our data is divided into 12 csv files, one for each month

We have to combine the files in one dataset so we can proceed with data exploration and analysis

but first let's pick a file and investigate it a little to see if some preprocessings are needed before combining the files, I chose **Sales_March_2019.csv**

In [4]:
# read csv file and print first 5  rows
march_df = pd.read_csv("./Sales_Data/Sales_March_2019.csv")
march_df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,162009,iPhone,1,700,03/28/19 20:59,"942 Church St, Austin, TX 73301"
1,162009,Lightning Charging Cable,1,14.95,03/28/19 20:59,"942 Church St, Austin, TX 73301"
2,162009,Wired Headphones,2,11.99,03/28/19 20:59,"942 Church St, Austin, TX 73301"
3,162010,Bose SoundSport Headphones,1,99.99,03/17/19 05:39,"261 10th St, San Francisco, CA 94016"
4,162011,34in Ultrawide Monitor,1,379.99,03/10/19 00:01,"764 13th St, San Francisco, CA 94016"


In [5]:
# check for duplicates
march_df[march_df.duplicated()].head(5)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
341,162332,Flatscreen TV,1,300,03/20/19 14:23,"925 10th St, Atlanta, GA 30301"
864,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
930,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1066,163018,AAA Batteries (4-pack),1,2.99,03/17/19 14:10,"694 Cedar St, Seattle, WA 98101"
1542,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
print("Number of rows with header values is {}\nNumber of rows full with nans is {}"\
      .format((march_df['Order ID']=='Order ID').sum()\
      ,sum([row.all() for row in march_df.isnull().values])))

Number of rows with header values is 35
Number of rows full with nans is 37


As we can see the data contains **duplicates** in addition to repeated **rows with header values** and **NaNs**

so the files need some preprocessing before concatenation

**first**: drop all rows full with NaNs

**second**: drop rows repeated headers

**third**: drop duplicates but keep one instance of them

let's investigate another file to make sure all the data suffer from the same faults, if so I will apply the same processing steps on all the files

In [7]:
# read csv file and print first 5  rows
september_df = pd.read_csv("./Sales_Data/Sales_September_2019.csv")
# check for duplicates
september_df[september_df.duplicated()].head(5)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
21,248171,USB-C Charging Cable,1,11.95,09/05/19 15:06,"705 Hill St, New York City, NY 10001"
574,NaN,NaN,NaN,NaN,NaN,NaN
658,248787,AA Batteries (4-pack),1,3.84,09/09/19 12:30,"705 Adams St, San Francisco, CA 94016"
776,NaN,NaN,NaN,NaN,NaN,NaN
1383,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print("Number of rows with header values is {}\nNumber of rows full with nans is {}"\
      .format((september_df['Order ID']=='Order ID').sum()\
      ,sum([row.all() for row in september_df.isnull().values])))

Number of rows with header values is 17
Number of rows full with nans is 40


Another file that contains rows with header values in addition to duplicate rows and rows full with NaNs
this could be due to something wrong happened while collecting data

I will make a pipeline of preprocessing steps and apply it on each sub dataset before concatinating it with our big and final data frame

In [9]:
def drop_NaN_rows(dataframe):
    # drop rows if all entries are NaN
    return dataframe.dropna(how = 'all')

def drop_repeated_headers(dataframe):
    # create a boolean mask to check for rows with header values
    mask = (dataframe==dataframe.columns.values).all(1)
    
    # return the dataframe without these rows
    return dataframe[~mask]

def drop_duplicates(dataframe):
    # drop duplicates but keep first instance
    return dataframe.drop_duplicates(keep='first')

def process_dataframe(dataframe):
     return dataframe.pipe(drop_NaN_rows)\
        .pipe(drop_repeated_headers)\
        .pipe(drop_duplicates)

In [10]:
piped = process_dataframe(september_df)

# check for successful processing
print("Number of duplicates: {}\nNumber of repeated headers: {}\nNumber of NaN rows: {}"
     .format(piped.duplicated().sum()\
             ,(piped==piped.columns.values).all(1).sum()\
             , piped.isnull().all().sum())
     )

Number of duplicates: 0
Number of repeated headers: 0
Number of NaN rows: 0


In [12]:
# check if the file already exists
files = os.listdir('./Sales_Data')

if 'data.csv' in files:
    df = pd.read_csv('./Sales_Data/all_data.csv')
else:
    
    # get names of dataset files of interest
    regex = re.compile("Sales_.*_2019.csv")
    files = list(filter(regex.match, files))

    # empty dataframe to concat the datasets
    df = pd.DataFrame() 

    for file in files:
        # read sub dataset
        small_df = pd.read_csv('./Sales_Data/'+file)
        
        # process the dataset
        small_df = process_dataframe(small_df)
        
        # extract month name : ex 'Sales_April_2019' => 'April'
        small_df['month'] = re.findall("(?:_)(\w+)(?:_)",file)[0]
        
        # concat the read data with the previous ones
        df = pd.concat([df, small_df], ignore_index=True)
    
    # store the combined datasets in a file for future runs
    df.to_csv('./Sales_Data/data.csv', index=False)

# Data Preprocessing

Now after reading and concatinating all files, it's time to investiate the data and do some preprocessing if it's needed before making insights about our market data

In [13]:
# display first 5 rows to get an idea about the data
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,month
0,162009,iPhone,1,700,03/28/19 20:59,"942 Church St, Austin, TX 73301",March
1,162009,Lightning Charging Cable,1,14.95,03/28/19 20:59,"942 Church St, Austin, TX 73301",March
2,162009,Wired Headphones,2,11.99,03/28/19 20:59,"942 Church St, Austin, TX 73301",March
3,162010,Bose SoundSport Headphones,1,99.99,03/17/19 05:39,"261 10th St, San Francisco, CA 94016",March
4,162011,34in Ultrawide Monitor,1,379.99,03/10/19 00:01,"764 13th St, San Francisco, CA 94016",March


In [14]:
# display information about our dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185686 entries, 0 to 185685
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          185686 non-null  object
 1   Product           185686 non-null  object
 2   Quantity Ordered  185686 non-null  object
 3   Price Each        185686 non-null  object
 4   Order Date        185686 non-null  object
 5   Purchase Address  185686 non-null  object
 6   month             185686 non-null  object
dtypes: object(7)
memory usage: 9.9+ MB


Dataframe **info** function provides us with information that helps to know more about column types in the dataframe in addition to know if there is any NaNs by comparing number of index entries found at the second line and number of Non-Null entries for each column

From the output it's obvious that:

* Some columns should have types of **int** or **float** but their type is **object** (string) instead, like: **Quantity Ordered** and **Price Each**

* Also **Order Date** should have datetime type so we can benefit from python datetime module


* For **Purchase Address** there is a lot of information in the address so it might be a good idea to extract **Street**, **State** and **City** from the address and use them in further analysis later

Also it might be more handy to create a new column represents total cost of the order by multiplying the two columns `Price Each` and `Quantity Ordered`, since dealing with total price is more logical

In [18]:
# Quantity Ordered should be integer 
df['Quantity Ordered'] = df['Quantity Ordered'].astype('int')

# for Price Each it has some values with floating point
# so it's better to change its type to float instead
df['Price Each'] = df['Price Each'].astype('float')

# chang Order Date column to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])


# Address consists mainly of three groups [street, state, city abbreviation]
# I used regex module to build a regex with comments for better understanding
# to capture only these three groups
regx = re.compile(r"""
           (?:\d+\s*)       # building number, we are not interested in it
           (?P<street>.*St) # Street Name
           (?:,\s*)         # remove ',' with any spacing
           (?P<state>.*)    # State Name
           (?:,\s*)         # remove ',' with any spacing
           (?P<city>\w{2})  # City abbreviation
           """, re.X)

# applying the regex
decoded_address = df['Purchase Address'].str.extract(regx)

# add the new columns to our dataframe
df = pd.concat([df, decoded_address], axis = 1)

# create the new column `total cost`
df['Total Cost'] = df['Quantity Ordered'] * df['Price Each']

In [20]:
# let's have a final look on the dataframe before proceeding with Data Exploration
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,month,street,state,city,street,state,city,Total Cost
0,162009,iPhone,1,700.00,2019-03-28 20:59:00,"942 Church St, Austin, TX 73301",March,Church St,Austin,TX,Church St,Austin,TX,700.00
1,162009,Lightning Charging Cable,1,14.95,2019-03-28 20:59:00,"942 Church St, Austin, TX 73301",March,Church St,Austin,TX,Church St,Austin,TX,14.95
2,162009,Wired Headphones,2,11.99,2019-03-28 20:59:00,"942 Church St, Austin, TX 73301",March,Church St,Austin,TX,Church St,Austin,TX,23.98
3,162010,Bose SoundSport Headphones,1,99.99,2019-03-17 05:39:00,"261 10th St, San Francisco, CA 94016",March,10th St,San Francisco,CA,10th St,San Francisco,CA,99.99
4,162011,34in Ultrawide Monitor,1,379.99,2019-03-10 00:01:00,"764 13th St, San Francisco, CA 94016",March,13th St,San Francisco,CA,13th St,San Francisco,CA,379.99


# Data Exploration

## Q1: What was the best month for sales

In [ ]:
# Now we are ready to go
# I will do it by calculating the average sales for each month and draw a histogram to visualize it

means = df.groupby('months')['sales'].agg('sum')
# Let's sort the values by the order of months 
means = means.reindex(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
               'October', 'November', 'December'])

plt.figure(figsize = (10,5))
plt.plot(means)
plt.xticks(rotation = 15)
plt.title("Months Total sales")
plt.xlabel("Months")
plt.ylabel("sales")
plt.grid()
plt.legend(['Total Sales'])
plt.show()

So according to the line plot above, the best month for sales was December with total sales of more than 4,500,000$
we also can see the trend in the sales, total sales was increasing starting from January till April, then started decreasing till it reached the least amount in September, and then started to increase again with the best sales to be in December

## Q1: What City had the highest number of sales?

In [ ]:
plt.figure(figsize = (10,5))

num_sales = df.groupby('city')['Order ID'].agg('count').plot(kind = 'bar')
plt.xticks(rotation = 25)
plt.title("Cities Total Orders")
plt.xlabel("Cities")
plt.ylabel("number of Orders")
plt.legend(["Orders"])
plt.show()

So the data we have contains orders for few unique Cities shown in the graph above
The City with the largest number of orders was San Francisco with more than 40000 orders, then Los Angeles with almost 30000 orders, etc.
Since this data is for electronics shop, it could be used to know in what cities the shop should have more branches to meet the market needs

## Q3: For each month, what is the most sold product

In [ ]:
# most sold product in each month
products = df.groupby('months')['Product'].value_counts().unstack()

In [ ]:
idx = 0
for idx in range(12):
    products.loc[products.index[idx], :].plot(kind = 'bar')
    plt.title(products.index[idx]+" Products")
    plt.show()

## Q3: What time should the shop display advertisments to maximize liklihood of cutomer's buying products?

In [ ]:
# for this question we create new column 'Hour' represents the hour the order was made
df['Hour'] = df['Order Date'].dt.hour
df.head()

In [ ]:
plt.figure(figsize=(10,5))
hours = df.groupby('Hour').groups.keys()
plt.plot(hours, df.groupby(['Hour']).count())
plt.xticks(list(hours))
plt.xlabel("Hours")
plt.ylabel("Number of Orders")
plt.title("Number of Orders for each hour of the day")
plt.grid()
plt.legend(['Number of Orders'])
plt.show()

The Hour column values are in 24-hour format, so 0 means 12:00 AM and 23 means 11:00 PM in 12-hour format
starting from 3:00 AM the customers often buy orders will 12:00 PM then number of orders decrease a little but then returned to increase till it's the maximum at 7:00 PM then it decreases
I can feel the trend in the data and the correlation between it and the work hours of the day
so the graph shows that if the shop website would display advertisments, it would be better to be at 7:00 PM since it's the time with maximum number of orders.

## Q4: What Products are most often sold together?

In [ ]:
# each order has Order ID, so I someone ordered more than one product at the same time, those products will have
# the same Order ID number, so to answer this question we will group the data by Order ID then count for each pair
# how often it was ordered

same_ordered = df[df['Order ID'].duplicated(keep = False)]

In [ ]:
# Create new Column Grouped to store products in the same order by joining them with ','
same_ordered['Grouped'] = same_ordered.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))

# now due to the new column we created in the dataframe, the Grouped values are repeated for each order in each group
# so we need to drop duplicates to avoid misleading results

grouped = same_ordered[['Order ID', 'Grouped']].drop_duplicates()
grouped.head(7)

In [ ]:
# To answer the question, I will use combinations to create combinations of k {for the sake of this question, k = 2}
# and use Counter to count number of occurrences of these combinations.

from itertools import combinations
from collections import Counter

count = Counter()

for row in grouped['Grouped']:
    row_list = row.split(',') # list of products for each Order
    count.update(Counter(combinations(row_list, 2))) # update the counter of the combinations of the products
    
for key, value in count.most_common(10):
    print(key,"Occurred ", str(value)+" time")

In [ ]:
top10 = count.most_common(10)
values = list([top10[i][1] for i in range(10)])
ticks = list([top10[i][0] for i in range(10)])
#plt.bar(ticks, values)

In [ ]:
# What product sold the most? Why do you think it's sold the most?

df.head()

In [ ]:
product_group = df.groupby('Product')
quntity_ordered = product_group.sum()['Quantity Ordered']
prices = df.groupby('Product').mean()['Price Each']


In [ ]:
fig, ax1 = plt.subplots(figsize = (10,5))
ax2 = ax1.twinx()
quntity_ordered.plot(kind = 'bar', ax = ax1)
prices.plot(ax = ax2, color = 'r')
ax1.set_ylabel("Count")
ax2.set_ylabel("Price")
plt.title("Quantity Ordered for each Product")
ax2.legend()
plt.show()